In [ ]:
!pip install -q transformers accelerate datasets  


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
# Load model directly
from transformers import AutoProcessor, AutoModelForImageTextToText

processor = AutoProcessor.from_pretrained("google/gemma-3-4b-it")
model = AutoModelForImageTextToText.from_pretrained("google/gemma-3-4b-it")

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset
from datasets import Image as dsImage

In [ ]:
import json


example_output = {
    "image_id": 100,
    "qa_id": 10674688,
    "url": "https://example.com/image.jpg",
    "question": "どんな天候ですか？",
    "pred_answer": "晴れの日",
    "gt_answer": "晴天"
}


print(json.dumps(example_output, ensure_ascii=False, indent=2))

if 'ds' in locals():
        first_example = next(iter(ds["test"]))
        print("\n実際のデータセット例:")
        print(f"質問: {first_example['question']}")
        print(f"回答: {first_example['answer']}")
        
        
        real_example = {
                "image_id": 1,
                "qa_id": 10000001,
                "url": first_example.get("image_url", "https://example.com/image.jpg"),
                "question": first_example["question"],
                "pred_answer": "モデルによる予測回答例",
                "gt_answer": first_example["answer"]
        }
        print("\n実データを基にした出力例:")
        print(json.dumps(real_example, ensure_ascii=False, indent=2))

In [4]:
ds = load_dataset("SakanaAI/JA-VLM-Bench-In-the-Wild")
ds

README.md:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/5.84M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['page_url', 'image_url', 'image', 'question', 'answer'],
        num_rows: 50
    })
})

In [5]:
import io
import os
import json
import torch
import PIL

from datasets import load_dataset, Image
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

# ========== 1) データセット読み込み (decode=False) ==========

dataset_name = "SakanaAI/JA-VLM-Bench-In-the-Wild"
ds = load_dataset(dataset_name)
ds = ds.cast_column("image", Image(decode=False))  
test_ds = ds["test"]  

# ========== 2) モデル/プロセッサの読み込み ==========

model_id = "google/gemma-3-4b-it"
print(f"Loading model: {model_id}")

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True,
)

# ========== 3) map 用の推論関数 ==========

def infer_on_example(example):
    """
    「SakanaAI/JA-VLM-Bench-In-the-Wild」は
      - page_url
      - image_url
      - image (dict: {'path': str, 'bytes': bytes})
      - question
      - answer
    のカラムを持つ。

    このexampleに対してモデル推論を行い、'pred_answer'を追加して返す。
    """

    page_url = example["page_url"]
    image_url = example["image_url"]
    question_text = example["question"]
    gt_answer = example["answer"]

    img_bytes = example["image"]["bytes"]  # decode=False なのでバイト列が入っている
    temp_path = None
    pred_answer = None

    # 画像バイト列があれば一時ファイルに書き出してモデルに与える
    if img_bytes is not None:
        temp_path = "temp_image.jpg"
        with open(temp_path, "wb") as f:
            f.write(img_bytes)

        # 推論用プロンプト作成
        messages = [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": "You are a helpful Japanese vision-language assistant."
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": temp_path}, 
                    {
                        "type": "text",
                        "text": f"質問: {question_text}\n\nURL: {image_url}\nページ: {page_url}"
                    }
                ]
            }
        ]

        # プロンプトの形式をモデル向けに変換
        inputs = processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(model.device)

        input_len = inputs["input_ids"].shape[-1]

        # 生成
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                max_new_tokens=128,
                do_sample=False
            )

        gen_part = generated[0][input_len:]
        decoded_answer = processor.decode(gen_part, skip_special_tokens=True)
        pred_answer = decoded_answer

        # 一時ファイル削除
        if os.path.exists(temp_path):
            os.remove(temp_path)

    # 推論結果を格納
    example["pred_answer"] = pred_answer

    
    return {k: v for k, v in example.items() if k != "image"}

# ========== 4) map の実行 (推論) ==========

test_inferred = test_ds.map(
    infer_on_example,
    batched=False,
    remove_columns=["image"]  # 明示的に削除しないと再エンコードエラーが出ることがある
)

# ========== 5) 結果書き出し ==========

output_file = "gemma3_inference.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for ex in test_inferred:
        # pred_answerはモデルの推論結果
        item = {
            "page_url": ex["page_url"],
            "image_url": ex["image_url"],
            "question": ex["question"],
            "gt_answer": ex["answer"],
            "pred_answer": ex["pred_answer"],
        }
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"All done! Saved in {output_file}")


Loading model: google/gemma-3-4b-it


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


All done! Saved in gemma3_inference.jsonl


In [ ]:
import io
import os
import json
import torch
import PIL
from datasets import load_dataset, Image
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

# ========== 1) Load dataset (decode=False) ==========

dataset_name = "SakanaAI/JA-VLM-Bench-In-the-Wild"
ds = load_dataset(dataset_name)
ds = ds.cast_column("image", Image(decode=False))  
test_ds = ds["test"]  

# ========== 2) Load model/processor ==========

model_id = "google/gemma-3-4b-it"
print(f"Loading model: {model_id}")

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id,
    trust_remote_code=True,
    device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True,
)

# ========== 3) Inference function for map ==========

def infer_on_example(example):
    """
    The "SakanaAI/JA-VLM-Bench-In-the-Wild" dataset has the following columns:
      - page_url
      - image_url
      - image (dict: {'path': str, 'bytes': bytes})
      - question
      - answer
    
    This function performs model inference on the example and adds 'pred_answer' before returning.
    """

    page_url = example["page_url"]
    image_url = example["image_url"]
    question_text = example["question"]
    gt_answer = example["answer"]

    img_bytes = example["image"]["bytes"]  # Contains byte sequences because decode=False
    temp_path = None
    pred_answer = None

    # If image bytes exist, write to a temporary file and provide to the model
    if img_bytes is not None:
        temp_path = "temp_image.jpg"
        with open(temp_path, "wb") as f:
            f.write(img_bytes)

        # Create prompt for inference
        messages = [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": "You are a helpful Japanese vision-language assistant."
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": temp_path}, 
                    {
                        "type": "text",
                        "text": f"質問: {question_text}\n\nURL: {image_url}\nページ: {page_url}"
                    }
                ]
            }
        ]

        # Convert prompt format for the model
        inputs = processor.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to(model.device)

        input_len = inputs["input_ids"].shape[-1]

        # Generate
        with torch.no_grad():
            generated = model.generate(
                **inputs,
                max_new_tokens=128,
                do_sample=False
            )

        gen_part = generated[0][input_len:]
        decoded_answer = processor.decode(gen_part, skip_special_tokens=True)
        pred_answer = decoded_answer

        # Delete temporary file
        if os.path.exists(temp_path):
            os.remove(temp_path)

    # Store inference results
    example["pred_answer"] = pred_answer

    
    return {k: v for k, v in example.items() if k != "image"}

# ========== 4) Execute map (inference) ==========

test_inferred = test_ds.map(
    infer_on_example,
    batched=False,
    remove_columns=["image"]  # Explicitly removed to avoid re-encoding errors
)

# ========== 5) Write results ==========

output_file = "gemma3_inference.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for ex in test_inferred:
        # pred_answer is the model's inference result
        item = {
            "page_url": ex["page_url"],
            "image_url": ex["image_url"],
            "question": ex["question"],
            "gt_answer": ex["answer"],
            "pred_answer": ex["pred_answer"],
        }
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"All done! Saved in {output_file}")

In [ ]:
import json

# Read the inference results file

# Translate the output and create a new file with English comments
with open("gemma3_inference.jsonl", "r", encoding="utf-8") as f_in, \
    open("gemma3_inference_en.jsonl", "w", encoding="utf-8") as f_out:
    
    for line in f_in:
       item = json.loads(line)
       
       # Create a new dictionary with translated field names
       en_item = {
          "page_url": item["page_url"],
          "image_url": item["image_url"],
          "question": item["question"],  # Keep original Japanese question
          "ground_truth_answer": item["gt_answer"],  # Keep original Japanese answer
          "predicted_answer": item["pred_answer"],  # Keep original Japanese answer
       }
       
       f_out.write(json.dumps(en_item, ensure_ascii=False) + "\n")

print("Translation complete! Saved in gemma3_inference_en.jsonl")

# You can also examine the first few records
with open("gemma3_inference_en.jsonl", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
       if i >= 3:  # Show first 3 examples
          break
       print(f"Example {i+1}:")
       print(json.dumps(json.loads(line), ensure_ascii=False, indent=2))
       print()